<h1> Main.ipynb </h1>
<h2> 1. Import libraries and modules </h2>

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


from src.Data_Retriever import DataRetriever as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

# from src.Optimization.Environment import PortfolioEnvironment as PorEnv
# from src.Optimization.RLModelCompilation import RL_Model as RLM
from src.Optimization.NeuralNet import CustomNeuralNet as CusNN
from src.Optimization.NeuralNet import CustomSACPolicy as CSACP

from src.Result.Menchero_OGA import MencheroOGA as MOGA
from src.Result.IndPortResults import GenerateResult as GR
from src.Result.OverviewResults import ResultConveyor as RC

<h2> 2. Define operating variables </h2>

In [4]:
# Define necessary non-fixed variables
trading_n = 30
history_usage = 100
n_sectors = 6
n_stocks_per_sector = 4

# For RL algorithm
history_usage_RL = 10

<h2> 3. Define stock dataframe </h2>

In [5]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", "SUBC.OL", "BWO.OL",]
ticker_df["Seafood (food)"] = ["ORK.OL", "MOWI.OL", "SALM.OL", "LSG.OL"]
ticker_df["Materials"] = ["NHY.OL", "YAR.OL", "RECSI.OL", "BRG.OL"]
ticker_df["Technologies"] = ["TEL.OL", "NOD.OL", "ATEA.OL", "BOUV.OL"]
ticker_df["Financial"] = ["STB.OL", "DNB.OL", "GJF.OL", "AKER.OL"]
ticker_df["Shipping"] = ["WAWI.OL", "SNI.OL", "BELCO.OL", "ODF.OL"]
ticker_df

,Petroleum,Seafood (food),Materials,Technologies,Financial,Shipping
0,EQNR.OL,ORK.OL,NHY.OL,TEL.OL,STB.OL,WAWI.OL
1,AKRBP.OL,MOWI.OL,YAR.OL,NOD.OL,DNB.OL,SNI.OL
2,SUBC.OL,SALM.OL,RECSI.OL,ATEA.OL,GJF.OL,BELCO.OL
3,BWO.OL,LSG.OL,BRG.OL,BOUV.OL,AKER.OL,ODF.OL


<h2> 4. Define ESG array </h2>

In [6]:
# Defining ESG scores for respective securities
esg_scores = [36.6, 35.3, 17.9, 18, 
              18, 21.2, 18.7, 29.2, 
              15.7, 25.6, 25.6, 18.4, 
              19.8, 13.8, 18.1, 19, 
              17.2, 14, 17.2, 19.5, 
              19.7, 21.2, 26.8, 19.3]

In [15]:
my_df = pd.read_csv("C:\Programmering\jakob\BBAN5000-Master-Thesis\Data\RL_weights_train.csv")

alfa = GR('train', n_sectors, n_stocks_per_sector, 100, esg_scores,ticker_df.columns)
alfa.friple_frequency_analysis()

[1.00541252 1.00857672 0.99980161 0.98394184 0.99149874 0.99341396
 0.98797465 0.98945287 0.98905267 0.98739737 0.98549693 0.98735756
 0.9957576  0.99313022 0.99138904 0.99510976 0.99007879 0.98727187
 0.97991789 0.97865248 0.9690425  0.97098534 0.96683304 0.96720824
 0.95515196 0.95898989 0.9621249  0.95608392 0.94794731 0.93942004
 0.9383651  0.93769302 0.92617051 0.92836249 0.92858916 0.92784484
 0.92665161 0.92571173 0.92167522 0.92395795 0.93535166 0.93095285
 0.94657311 0.95644168 0.94741146 0.94683888 0.94392657 0.94693901
 0.94559876 0.94729888 0.9432707  0.94804693 0.95785266 0.96158143
 0.95946333 0.9576457  0.94395086 0.94630495 0.94364232 0.94438689
 0.94872627 0.94655576 0.94445229 0.94296913 0.93937541 0.94339087
 0.95077077 0.95370406 0.95791577 0.95884703 0.96075456 0.9626891
 0.95263639 0.95469101 0.95631434 0.95548497 0.95266667 0.94938083
 0.96309437 0.9631236  0.96440679 0.9648344  0.97141147 0.96957885
 0.9582053  0.95339946 0.93998276 0.90417052 0.91710479 0.90752

In [43]:
my_df = pd.read_csv("C:\Programmering\jakob\BBAN5000-Master-Thesis\Data\RL_weights_test.csv")

alfa = GR('test', n_sectors, n_stocks_per_sector, 400, esg_scores,ticker_df.columns)
alfa.friple_frequency_analysis()

[1.00675545 1.00707545 1.01228491 1.01118069 1.00694221 1.00715331
 1.00757184 1.0096245  1.01065256 1.01234391 1.01382955 1.01423269
 1.00728747 1.01789305 1.01796704 1.01431046 1.01968806 1.02069172
 1.02434704 1.02212142 1.02418225 1.02584249 1.03198088 1.03811809
 1.04075003 1.04142291 1.04497456 1.04579035 1.04957901 1.04927134
 1.05479082 1.05759971 1.06027811 1.06092925 1.06251479 1.06482633
 1.06218435 1.06608708 1.0644842  1.0755631  1.08334031 1.08251243
 1.07650567 1.06972977 1.067461   1.06794861 1.07173681 1.0696888
 1.06610639 1.06976284 1.06969728 1.07146324 1.06966011 1.07007929
 1.07493115 1.09891756 1.10364274 1.10321474 1.11198653 1.11305183
 1.11750997 1.11907505 1.11944639 1.12152092 1.12545306 1.12353721
 1.12022076 1.12123877 1.12511081 1.1275411  1.13201094 1.13263736
 1.13099184 1.13641035 1.14774728 1.15268693 1.15928486 1.15587888
 1.15516898 1.15403058 1.15438067 1.15750638 1.14970874 1.15105208
 1.1536852  1.15808052 1.15539884 1.16227872 1.15386352 1.15902

<h2> 5.-7. (Non-necessary for reproducability) </h2>
<h3>Data retrieval</h3>
<h3>Benchmark optimization (MPT)</h3>
<h3>RL optimization x4</h3>
<br>
All of the data has been generated an stored in csvs. Therefore, I can comment out the code lines below.

In [5]:
# # Retrieve data from yf API: y-m-d
# data = DatRet(ticker_df, "2013-01-01", "2024-01-01")
# # In function below, set log=True to check for data availability
# data.retrieve_data()

In [6]:

# # Generate benchmark weights thorugh MPT using Sharpe ratio
# benchmark = MPT(history_usage, trading_n)
# # IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
# benchmark.frequency_optimizing()

In [7]:
# objectives = ["Return", "Sharpe", "Sortino", "Sterling", "Return", "Sharpe", "Sortino", "Sterling"]
# esg_compliancy = [True, True, True, True, False, False, False, False]

# for i in range(len(objectives)):
#     reinforcement = RLM(esg_scores, 
#                         objective=objectives[i],
#                         window_size=history_usage_RL,
#                         total_timesteps=5000,
#                         esg_compliancy=esg_compliancy[i],
#                         )
#     reinforcement.train_model()
#     reinforcement.test_model()

<h2> 8. Attribution analysis </h2>

In [8]:
paths = ["Return_esg_True", "Sharpe_esg_True",
         "Sortino_esg_True","Sterling_esg_True",
         "Return_esg_False", "Sharpe_esg_False",
         "Sortino_esg_False","Sterling_esg_False",]

analysis_list = []
for i in range(len(paths)):
    att_anal = GR(paths[i],
            n_sectors, n_stocks_per_sector,
            trading_n,
            esg_scores, 
            ticker_df.columns)
    att_anal.friple_frequency_analysis()
    analysis_list.append(att_anal)

----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----


<h2> X. Ad hoc analysis </h2>

In [9]:
theta = RC(analysis_list, trading_n)
theta.convey_results()